In [15]:
from cv2 import line, putText, FONT_HERSHEY_SIMPLEX
import numpy as np


def draw_tracks(sequence : np.array, tracks_data : np.array) -> np.array:
    '''
    Function that draw the tracks given the sequence and the array of tracks
    
    Input:
        - squence: array of shape (N,H,W,C)
            N: number of frames
            H: Height
            W: Width
            C: Channel (empty if grayscale)
        - tracks_data: array where:
                            tracks_data[0] track_id
                            tracks_data[1] x
                            tracks_data[2] y
                            tracks_data[3] fluorescence
                            tracks_data[4] frame
    
    Ouput:
        - seq_out: array of shape (N,H,W,C) 
    '''
    frames, H, W = sequence.shape[0:3]
    seq_out = sequence.copy()
    
    if len(sequence.shape) < 4:
        seq_out = np.zeros((frames, H, W, 3))
        seq_out[:, :, :, 0] = sequence.copy()
        
    tracks_id = (np.unique(tracks_data[:, 0])).tolist()
    
    font                   = FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10,500)
    fontScale              = 0.5
    fontColor              = (255,255,255)
    lineType               = 1

    
    tracks = []
    for track_id in tracks_id:
        data_track = tracks_data[tracks_data[:, 0]==track_id]
        coord = np.ones((frames, 4)) * -1
        coord[np.uint(data_track[:, 4])] = data_track[:, 0:4]
        tracks.append(coord)
    colors = np.random.uniform(0, 255, (len(tracks_id), 3))
    
    for frame in range(frames):
        start = 1
        if frame > 20:
            start = frame - 20
        for i in range(start,frame+1):
            
            for tr, track in enumerate(tracks):
                if track[i,1]>0 and track[i-1,1]>0:
                    line(seq_out[frame], (int(track[i-1,1]),int(track[i-1,2]) ), (int(track[i,1]),int(track[i,2]) ), colors[tr])
            
                    fontColor = colors[tr]
                    coord = (int(track[frame,1]) + 5,int(track[frame,2]) + 5)
                    '''putText(seq_out[frame],'ID: %d, F : %d' % (track[frame,0], track[frame,3]), 
                            coord, 
                            font, 
                            fontScale,
                            fontColor,
                            lineType)'''
                    
    return np.uint8(seq_out)


In [18]:
import pandas as pd
import tifffile
from imageio import mimwrite as mp4_writer

in_sequence = tifffile.TiffFile("dataset_1(40Hz)_segmented_.tif")
sequence = in_sequence.asarray()
data = pd.read_csv("dataset_1(40Hz)_data.csv")
data = (data.to_numpy())[:, 0:5]
#data_in = np.zeros((data.shape[0], 5))
#data_in[:, (0, 1, 2, 4)] = data
seq_tracks = draw_tracks(sequence, data)
mp4_writer("output.mp4", seq_tracks, format="mp4", fps=5.)


In [8]:
data = pd.read_csv("out_fluorescencia_c.csv")

In [14]:
data

,Unnamed: 0,id,x,y,fluorescence,frame
0,0,0,428,416,157,7
1,2,0,428,416,158,8
2,7,0,428,416,165,9
3,14,0,428,416,156,10
4,27,0,428,416,160,11
...,...,...,...,...,...,...
4971,4859,87,375,427,131,117
4972,4916,87,376,432,138,118
4973,4974,87,373,436,148,119
4974,4917,88,396,14,155,118
